## Aula 05 - Filtragem Híbrida - Exercícios

In [1]:
import pandas as pd
import numpy as np

### Importar base de dados

In [72]:
import wget
!python3 -m wget https://github.com/mmanzato/MBABigData/raw/master/ml-20m-compact.tar.gz
!tar -xvzf ml-20m-compact.tar.gz

100% [....................................................] 65019041 / 65019041
Saved under ml-20m-compact.tar.gz
x dataset/
x dataset/tags_sample.csv
x dataset/.DS_Store
x dataset/movies_sample.csv
x dataset/genome-tags.csv
x dataset/ml-youtube.csv
x dataset/genome-scores.csv
x dataset/ratings_sample.csv


### Obs. Nesta aula, você poderá escolher entre implementar o exercício 1 **OU** o exercício 2.

***Exercício 01:*** Implemente uma hibridização monolítica/combinação usando a seguinte heurística:
- Uso do algoritmo ItemAtributeKNN, sendo a hibridização feita no cálculo das similaridades entre os itens.
- Se a quantidade de usuários que avaliaram ambos os itens for maior que um limiar L1, calcule a similaridade entre esses itens usando cosseno aplicado à representação baseada em notas.
- Caso contrário, calcule a similaridade entre os itens usando metadados (gêneros por exemplo).  

Compare os resultados do algoritmo híbrido com as versões isoladas do mesmo algoritmo.

In [ ]:
# TODO

***Exercício 02:*** Vamos implementar um recomendador híbrido canalizado em cascata, no cenário de ranqueamento. A ideia é que um primeiro algoritmo gere uma lista C1 de N=50 itens candidatos à recomendação para cada usuário. Em seguida, um outro recomendador irá gerar uma outra lista C2 também de N=50 itens candidatos à rcomendação para cada usuário. Por fim, o ranking final será a intersecção entre C1 e C2, sendo o score de cada itens formado pela média aritmética dos scores de cada lista. Avalie o desempenho.

Dica 1: utilize o parâmetro rank_length disponível nos algoritmos de ranqueamento do CaseRecommender para especificar o tamanho N de recomendações para cada usuário.

Dica 2: você pode gravar num arquivo os rankings gerados por um algoritmo para cada usuário especificando o nome do arquivo no parâmetro output_file.

Dica 3: consulte a Aula 04 que contém algumas métricas de avaliação de ranqueamento. Como você irá gerar o ranking final externamente ao CaseRecommender, será necessário avaliá-lo usando funções próprias.

In [ ]:
# TODO